In [1]:
import torch
import torch.nn as nn
from models.awpooling import *
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.tensorboard import SummaryWriter
from models.simplenet import SimpleNet
from models.vggaw import VGG11AW, VGG11AWT
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.DoubleTensor)
torch.manual_seed(777)
delta = torch.tensor(1e-8, device=device)

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_path = '/root/notebooks/nfs/work/dataset/tiny-imagenet-200'
traindir = os.path.join(data_path, 'train')
valdir = os.path.join(data_path, 'val')

writer = SummaryWriter(f'/root/notebooks/nfs/work/larry.lai/AWPooling/Gradient_analysis/Double_fixed/{delta}')
train_ds = ImageFolder(
    root=traindir, 
    transform=transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])
)

val_ds = ImageFolder(
    root=valdir,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)
train_loader = torch.utils.data.DataLoader(train_ds, shuffle=True, batch_size=128, num_workers=2, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=128, num_workers=2, pin_memory=True)

model = SimpleNet()

params = [{'params': p, 'lr': 0.1} for n, p in model.named_parameters() if 'aw' not in n]
optimizer = torch.optim.SGD(params)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
model.to(device)

SimpleNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (aw1): AWPool2d_()
  (globalavg): AdaptiveAvgPool2d(output_size=1)
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=200, bias=True)
  )
)

In [3]:
# training process
for i, data in enumerate(train_loader):
    images, label = data
    images = images.to(device)
    label = label.to(device)

    logits = model(images)
    loss = criterion(logits, label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# validation process
for i, data in enumerate(val_loader):
    # freeze batchNorm layer and Dropout layer
    model.eval()

    images, label = data
    images = images.to(device)
    label = label.to(device)

    # caculate numercial difference
    with torch.no_grad():
        model.aw1.t.requires_grad = False
        origin_t = model.aw1.t.item()

        # forward
        model.aw1.t.copy_(torch.tensor(origin_t, dtype=torch.double) + delta)
        print(f'forward temperature {model.aw1.t}')
        logits = model(images)
        forward_loss = criterion(logits, label)

        # backward
        model.aw1.t.copy_(torch.tensor(origin_t, dtype=torch.double) - delta)
        print(f'backward temperature {model.aw1.t}')
        logits = model(images)
        backward_loss = criterion(logits, label)

    model.aw1.t.copy_(torch.tensor(origin_t, dtype=torch.double))

    # Automatic differentiation
    model.aw1.t.requires_grad = True
    logits = model(images)
    base_loss = criterion(logits, label)
    base_loss.backward()

    exact_diff = model.aw1.t.grad
    f_diff = (forward_loss - base_loss) / delta
    b_diff = (base_loss - backward_loss) / delta
    c_diff = (forward_loss - backward_loss) / (2 * delta)

    print(f'iter {i}: ' \
            f'exact grad: {exact_diff.item(): .8f}\n' \
            f'forward diff: {f_diff.item(): .8f}\n'  \
            f'backward diff: {b_diff.item(): .8f}\n'  \
            f'central diff: {c_diff.item()}\n')

    # plot gradient on tensorboard
    writer.add_scalars('Gradient analysis', {
        'Automatic differentiation': exact_diff,
        'Forward difference': f_diff,
        'Backward difference': b_diff,
        'Central difference': c_diff,
    }, i)

    f_diff = torch.abs(exact_diff - f_diff) if torch.sign(exact_diff) == torch.sign(f_diff) else -torch.abs(exact_diff - f_diff)
    b_diff = torch.abs(exact_diff - b_diff) if torch.sign(exact_diff) == torch.sign(b_diff) else -torch.abs(exact_diff - b_diff)
    c_diff = torch.abs(exact_diff - c_diff) if torch.sign(exact_diff) == torch.sign(c_diff) else -torch.abs(exact_diff - c_diff)

    writer.add_scalars('Difference', {
        'Forward': f_diff,
        'Backward': b_diff,
        'Central': c_diff
    }, i)

    # update model parameters
    optimizer.step()
    optimizer.zero_grad()
    model.aw1.t.grad.zero_()

print(f'{delta} done!')

forward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
backward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
iter 0: exact grad:  3.71662182
forward diff:  0.12290107
backward diff:  0.12290098
central diff: 0.12290102269219005

forward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
backward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
iter 1: exact grad: -0.38978911
forward diff: -0.38978900
backward diff: -0.38978909
central diff: -0.3897890454851449

forward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
backward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
iter 2: exact grad: -0.18794448
forward diff: -0.18794433
backward diff: -0.18794450
central diff: -0.18794441558611652

forward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
backward temperature Parameter containing:
tensor([1.0000], device='cuda:0')
iter 3: exact grad: -0.0009